In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import imgaug as ia
from imgaug import augmenters as iaa
from CA1_Yolo import *
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
img_dir = "../data/image/"
anno_dir = "../data/annotation/"

In [3]:
all_anno, labels = read_annotation_files(img_dir, anno_dir)

In [4]:
anchor_boxes = generateAnchorBoxes(all_anno, labels)

iter: 1 distances: 1860.3052192572877
iter: 2 distances: 121.49778664149235
iter: 3 distances: 59.042061824031805
iter: 4 distances: 58.40302839650466
iter: 5 distances: 56.92584721223961
iter: 6 distances: 46.34951987369291
iter: 7 distances: 66.89793894878895
iter: 8 distances: 24.88504954209361
iter: 9 distances: 11.726071341947048
iter: 10 distances: 10.159927459572462
iter: 11 distances: 3.153221802480845
iter: 12 distances: 2.599840755436628
iter: 13 distances: 8.556117950101473
iter: 14 distances: 9.912229387343391
iter: 15 distances: 5.1142279010256475


In [5]:
anchor_boxes

[28, 22, 37, 57, 41, 28, 55, 104, 57, 38, 70, 52, 91, 74, 104, 109, 114, 36]

In [6]:
train_size=0.8
train_valid_split = int(0.8 * len(all_anno))
np.random.shuffle(all_anno)
train_anno = all_anno[:train_valid_split]
valid_anno = all_anno[train_valid_split:]
max_boxes = max([len(anno['object']) for anno in (train_anno + valid_anno)])

In [7]:
len(train_anno)

97

In [8]:
train_generator = DataGenerator(
    annotations=train_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True,
    augment=True)

In [9]:
valid_generator = DataGenerator(
    annotations=valid_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True,
    augment=True)

In [10]:
train_model, infer_mode = YoloV3(
    numcls=len(labels),
    anchors=anchor_boxes,
    max_grid=[416, 416],
    batch_size=2,
    threshold=0.5,
    max_boxes=max_boxes)







Instructions for updating:
Use `tf.cast` instead.



In [11]:
train_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

In [12]:
def dummy_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_sum(y_pred))

In [13]:
opt = Adam(lr=1e-4, clipnorm=0.001)
train_model.compile(loss=dummy_loss, optimizer=opt)

In [14]:
callback = create_callbacks()

In [15]:
history = train_model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    verbose=1,
    steps_per_epoch=len(train_generator) * 3,
    epochs=300,
    callbacks=callback
)

Epoch 1/300
 20/144 [===>..........................] - ETA: 4:04 - loss: 111.0035 - yolo_loss_layer_1_loss: 14.2296 - yolo_loss_layer_2_loss: 30.2428 - yolo_loss_layer_3_loss: 64.2065

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[512,256,3,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_41/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss/yolo_loss_layer_3_loss/Mean_2/_2935]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[512,256,3,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_41/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ig